In [ ]:
# Image Scrapper - Lot of images need to be trained for deep learning models.
#                  So to collect images we need to do scrapping.

In [4]:
#pip install selenium

In [19]:
import os
import time
import requests
from selenium import webdriver

In [20]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)
        
    # build Google Query
    # {q} - The search element is passed as 'q'.
    
    search_url = "http://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&gs_I=img"
    
    # load the page
    wd.get(search_url.format(q = query))
    
    image_urls = set()
    image_count = 0
    result_start = 0
    
    while image_count < max_links_to_fetch:
        
        scroll_to_end(wd)
        
        # get all images thumbnail
        
        thumbnail_results = wd.find_elements_by_css_selector('img.Q4LuWd')
        number_results = len(thumbnail_results)
        
        print(f"Found : {number_results} search results. Extracting links from {result_start}:{number_results}")
        
        for img in thumbnail_results[result_start:number_results]:
            
            # try to click on every thumbnail so that we can get the image behind it
            
            try:
                
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue
                
            # Extract Image urls
            
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
                    
            image_count = len(image_urls)
            
            if len(image_urls) >= max_links_to_fetch:
                print(f"Found : {len(image_urls)} image links, done!")
                break
        else:
            print(f"Found: ", len(image_urls),"image links, looking for more")
            time.sleep(30)
            return
            load_more_button = wd.find_elements_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")
                
        # move the results startpoint further down
        
        results_starts = len(thumbnail_results)
        
    return image_urls    

In [21]:
def persist_image(folder_path: str, url:str, counter):
    
    try:
        image_content = requests.get(url).content
    
    except Exception as e:        
        print(f"Error - Could not download {url} - {e}")
    
    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb' )
        f.write(image_content)
        f.close()
        
        print(f"Success-saved {url} -as {folder_path}")
    except Exception as e:
        print(f"ERROR- Could not save {url} - {e}")
        
        
    

In [22]:
def search_and_download(search_term: str, driver_path:str, target_path = './images', number_images=10):
    
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split('.'))) # Make the image name inside images with the search string
    
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        
    with webdriver.Chrome(executable_path = driver_path) as wd:
        res = fetch_image_urls(search_term, number_images,wd = wd, sleep_between_interactions=0.5)
        
    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

In [23]:
DRIVER_PATH = r'C:\Users\pRoJyot\Downloads\Programs\chromedriver.exe'
search_term = 'apple'

search_and_download(search_term=search_term, driver_path=DRIVER_PATH)



Found : 100 search results. Extracting links from 0:100
Found : 10 image links, done!
Success-saved https://cdn1.vectorstock.com/i/1000x1000/96/55/red-apple-vector-679655.jpg -as ./images\apple
Success-saved https://thumbor.forbes.com/thumbor/fit-in/416x416/filters%3Aformat%28jpg%29/https%3A%2F%2Fspecials-images.forbesimg.com%2Fimageserve%2F5c13d14731358e5b4339c564%2F0x0.jpg%3Fbackground%3D000000%26cropX1%3D0%26cropX2%3D416%26cropY1%3D0%26cropY2%3D416 -as ./images\apple
Success-saved https://asset.swarovski.com/images/$size_1450/t_swa103/b_rgb:fafafa,c_scale,dpr_3.0,f_auto,w_500/5491974_png/red-apple-swarovski-5491974.png -as ./images\apple
Success-saved https://yt3.ggpht.com/a/AATXAJzLJovKyZ7JLdndJ5foqqFL-Xt8Zm28UtEuFRW2Gg=s900-c-k-c0xffffffff-no-rj-mo -as ./images\apple
Success-saved https://pbs.twimg.com/profile_images/1283958620359516160/p7zz5dxZ_400x400.jpg -as ./images\apple
Success-saved https://ichef.bbci.co.uk/news/1024/cpsprodpb/692A/production/_112922962_apple.jpg -as ./imag